In [2]:
import requests
import pandas as pd
import pprint
from datetime import date
import json
import glob

## Top Player Penalty Minutes

In [3]:
df = pd.DataFrame()
for file in glob.glob('games/2024*.json'):
    #print(file)
    with open(f'{file}') as json_file:
        data = json.load(json_file)
    dataGame = pd.json_normalize(data,record_path= ['plays'])
    dataGame['game']=file
    df=pd.concat([df,dataGame])
# df.describe()

# Penalties
# df=df[['typeDescKey','details.xCoord','details.yCoord','details.committedByPlayerId','typeCode','details.duration']].loc[df['typeDescKey']=='penalty']
penaltyColumns = [ 
    'typeDescKey',
    'details.xCoord',
    'details.yCoord',
    'details.committedByPlayerId',
    'details.drawnByPlayerId',
    'typeCode',
    'details.duration',
    'details.descKey',
    'periodDescriptor.number',
    'timeInPeriod'
]
df=df.loc[df['typeDescKey']=='penalty']

# df.groupby(df['details.committedByPlayerId'])
players=pd.read_csv('games/players.csv')
df=df.rename(columns={"details.committedByPlayerId":'playerId'})
df=pd.merge(df,players,on="playerId",how='outer')

df=df.loc[df['teamId']==12]
# df=df.loc[df['details.duration']!=0]
df.dropna(subset=['details.duration'],inplace=True)
penaltyPlayers = df.groupby(['lastName.default','firstName.default'])['details.duration'].sum()
penaltyPlayers = penaltyPlayers.sort_values(ascending=False)

print(penaltyPlayers)

lastName.default  firstName.default
Svechnikov        Andrei               18.0
Kotkaniemi        Jesperi              16.0
Aho               Sebastian            12.0
Walker            Sean                 12.0
Chatfield         Jalen                10.0
Gostisbehere      Shayne               10.0
Carrier           William               8.0
Blake             Jackson               8.0
Jarvis            Seth                  6.0
Orlov             Dmitry                6.0
Staal             Jordan                6.0
Burns             Brent                 6.0
Necas             Martin                4.0
Robinson          Eric                  4.0
Roslovic          Jack                  4.0
Drury             Jack                  2.0
Slavin            Jaccob                2.0
Martinook         Jordan                2.0
Name: details.duration, dtype: float64


## Top types of penalties that hurricanes perform

In [4]:
penaltyTypes = df.groupby(['details.descKey'])['details.descKey'].count()
penaltyTypes = penaltyTypes.sort_values(ascending=False)
print(penaltyTypes)
# df.info()
# df['details.descKey'].unique()

details.descKey
tripping                         15
hooking                          11
slashing                          7
delaying-game-puck-over-glass     6
cross-checking                    5
roughing                          5
high-sticking                     4
holding                           4
interference                      4
high-sticking-double-minor        3
holding-the-stick                 1
Name: details.descKey, dtype: int64


## Hurricane Penalties by Location

In [5]:
# penaltytype='tripping'

penaltyLocations = df.groupby(['lastName.default']).agg(
    avg_x=('details.xCoord','mean'),
    avg_y=('details.yCoord','mean'),
    std_x=('details.xCoord','std'),
    std_y=('details.yCoord','std'),
    count=('lastName.default','count'),
    duration=('details.duration','sum'))
penaltyLocations = penaltyLocations.sort_values(by='count',ascending=False)
print(penaltyLocations)

                      avg_x      avg_y      std_x      std_y  count  duration
lastName.default                                                             
Svechnikov        -1.666667   3.666667  62.391907  26.386550      9      18.0
Aho              -24.166667   9.666667  75.258001  24.654952      6      12.0
Kotkaniemi        -3.500000 -20.166667  82.332861  16.277797      6      16.0
Gostisbehere     -17.400000  -7.200000  79.396473  24.190907      5      10.0
Walker           -10.600000 -15.000000  58.028441  37.040518      5      12.0
Chatfield         28.000000  -8.000000  68.110205  27.120103      5      10.0
Carrier          -13.500000  18.500000  80.189359  21.079216      4       8.0
Blake             64.750000   6.500000  27.885181  32.562760      4       8.0
Jarvis            72.000000 -10.333333  15.099669  36.018514      3       6.0
Orlov              7.666667  -8.666667  89.271123  27.319102      3       6.0
Staal            -11.666667  -4.000000  88.669799  29.512709    

## types of records

In [12]:
df["typeDescKey"].unique()
#dataGame.dtypes

array(['penalty', nan], dtype=object)

## Goals

In [13]:
goals=df[(df["typeDescKey"]=="missed-shot") | (df["typeDescKey"]=="blocked-shot")]
# grouped=goals.groupby('team.name')
goals.head(3)[['timeInPeriod','details.reason']]
#goals[['team.name','coordinates.x','coordinates.y']]
# goals.describe



,timeInPeriod,details.reason


GET https://statsapi.web.nhl.com/api/v1/teams | Returns a list of data about
all teams including their id, venue details, division, conference and franchise information.
GET https://statsapi.web.nhl.com/api/v1/teams/ID | Returns the same information as above just
for a single team instead of the entire league.

Team Modifiers
Add these to the end of the url
?expand=team.roster | Shows roster of active players for the specified team
?expand=person.names | Same as above, but gives less info.

In [14]:
baseUrlEn="https://statsapi.web.nhl.com/api/v1/teams/12"
baseUrlEn="https://api-web.nhle.com/v1/roster-season/CAR"
# baseUrlEn="https://api.nhle.com/v1/roster/CAR/now"
response=requests.get(baseUrlEn)
dfroster=pd.json_normalize(response.json())
# print(response)
dfroster
skater=requests.get('https://api.nhle.com/stats/rest/en/skater/summary?8474150')
print(skater)


<Response [500]>


In [15]:
skater

<Response [500]>

In [17]:
df.head()

,game,eventId,timeInPeriod,timeRemaining,situationCode,homeTeamDefendingSide,typeCode,typeDescKey,sortOrder,periodDescriptor.number,...,details.assist1PlayerId,details.assist1PlayerTotal,details.assist2PlayerId,details.assist2PlayerTotal,details.servedByPlayerId,teamId,sweaterNumber,positionCode,firstName.default,lastName.default
0,games/2024020254.json,381.0,13:42,06:18,1551,left,509.0,penalty,205.0,1.0,...,NaN,NaN,NaN,NaN,NaN,12.0,20.0,C,Sebastian,Aho
1,games/2024020280.json,931.0,11:04,08:56,1551,right,509.0,penalty,669.0,3.0,...,NaN,NaN,NaN,NaN,NaN,12.0,20.0,C,Sebastian,Aho
2,games/2024020084.json,674.0,13:58,06:02,1551,right,509.0,penalty,448.0,2.0,...,NaN,NaN,NaN,NaN,NaN,12.0,20.0,C,Sebastian,Aho
3,games/2024020135.json,389.0,14:24,05:36,1551,left,509.0,penalty,208.0,1.0,...,NaN,NaN,NaN,NaN,NaN,12.0,20.0,C,Sebastian,Aho
4,games/2024020020.json,895.0,04:57,15:03,1451,right,509.0,penalty,593.0,3.0,...,NaN,NaN,NaN,NaN,NaN,12.0,20.0,C,Sebastian,Aho
